<a href="https://colab.research.google.com/github/Edward-TL/amazon_scraper/blob/master/EDITOR_urls_MasterDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#imports

In [2]:
#For the system
import os

#Manage of time
from datetime import datetime, timedelta
from pytz import timezone
import time
import re

#Manage of files
import pandas as pd
import csv

#scrap
from bs4 import BeautifulSoup
from openpyxl.workbook import Workbook
import requests

In [3]:
%cd 'drive/My Drive/Colab Notebooks/4SS/4SS_db/testing/Edition Test'
!ls

/content/drive/My Drive/Colab Notebooks/4SS/4SS_db/testing/Edition Test
br


#Centered Len

In [4]:
def centered_len(max_len):
    def centered_string(string):
        if type(string) != 'string':
            string = str(string)
            
        rest = max_len - len(string)
        n_spaces = int(rest/2)

        string_spaces = ' ' * n_spaces
        
        centered_string = string_spaces + string + string_spaces

        if len(centered_string)<max_len:
            centered_string = ' ' + centered_string

        return centered_string
    return centered_string

##Ubicación

In [12]:
%cd '..'
%cd 'Masters'
!ls

/content/drive/My Drive/Colab Notebooks/4SS/4SS_db/testing
/content/drive/My Drive/Colab Notebooks/4SS/4SS_db/testing/Masters
br  master_db_logs.csv	mx


##Corrección de fechas y Ranking

In [6]:
def preload_dbs(dir):

    '''Recive a directory with a group of dbs and returns an info array
    info[0] = array of DB's names
    info[1] = largest_len_db, for centered'''
    
    databases = os.listdir(dir)
    n_databases = len(databases)

    #initialize
    df = [None]*n_databases
    largest_len_db = 0
    n = 0

    for file_name in databases:
        db_name = f'{file_name}'
        df[n] = db_name
        if len(db_name) > largest_len_db:
            largest_len_db = len(db_name)

        n = n + 1

    info = [df, largest_len_db]
    return info

In [7]:
def print_info_headers(len: int):
    largest_len = info[1]
    centered_db = centered_len(largest_len)
    centered_header = centered_db('masterdbs')
    dashes = '-'*largest_len

    print(f' country | {centered_header}  | Edition')
    print(f' ------- | {dashes} | ------ ')

In [14]:
def replace_urls_with_ids(db_to_edit):
    #We don't need them now
    df = pd.read_parquet(db_to_edit)
    df.dropna(thresh=3)

    #Extraction of ID's
    df['Product ID B'] = df['Product links'].str.extract(r'(?<=dp\/)(.+)(?=\/ref)')
    df['Product ID A'] = df['Product links'].str.extract(r'(?<=dp\/)(.+)(?=\?)')
    
    df['Product ID B'].fillna(df['Product ID A'], inplace=True)

    df['Product ID'] = df['Product ID B']

    df['Image ID'] = df['Image urls'].str.extract(r'(?<=I\/)(.*)(?=\.jpg)')

    #Drop Other columns
    df.drop(columns=['Product links', 'Image urls'], inplace=True)

    # Reorder de DF
    df = df[['time', 'Rank', 'Product Names', 'Product ID', 'Image ID',
            'Stars', 'Reviews', 'Authors/Company', 'Edition/Console',
            'Price_std_or_min', 'Max_prices']]
    
    return df

In [9]:
# @replace_urls_with_ids
# def edit_function(db_to_edit):
#     return

In [10]:
def main_edition(info, country=None):

    centered_db = centered_len(info[1])

    df_names = info[0]
    
    for n in range(len(df_names)):

        if country:
            print(f'   {country}   ', end=' | ')

        #file name extract
        parquet_len = len('.parquet')
        file_name_len = len(df_names[n])-parquet_len
        file_name = df_names[n][0:file_name_len]

        #File name print to log
        print(f' {centered_db(file_name)}', end=' | ')

        #--------Edition--------
        #Load Dataframe
        db_to_edit = f'{dir}/{df_names[n]}'

        edited_db = replace_urls_with_ids(db_to_edit)
        #--------Upadting--------
        #File names
        csv_file = f'{country}/csv/{file_name}.csv'
        excel_file = f'{country}/excel/{file_name}.xlsx'
        parquet_file = f'{country}/parquet/{file_name}.parquet'
        #File updating
        edited_db.to_csv(csv_file,sep="|")
        edited_db.to_excel(excel_file, index=False)
        edited_db.to_parquet(parquet_file, allow_truncated_timestamps=True)

        print('Done')

## Deploy


In [15]:
secure = True

if secure == True:
    countries = ['mx', 'br']

    for country in countries:
        dir = f'{country}/parquet'
        info = preload_dbs(dir)
        print_info_headers(info[1])
        main_edition(info, country)

 country |                 masterdbs                 | Edition
 ------- | ---------------------------------------- | ------ 
   mx    |            mx-master_db_handmade          | Done
   mx    |              mx-master_db_hpc             | Done
   mx    |            mx-master_db_grocery           | Done
   mx    |          mx-master_db_electronics         | Done
   mx    |              mx-master_db_dvd             | Done
   mx    |          mx-master_db_digital-text        | Done
   mx    |             mx-master_db_books            | Done
   mx    |              mx-master_db_baby            | Done
   mx    |           mx-master_db_automotive         | Done
   mx    |         mx-master_db_amazon-devices       | Done
   mx    |            mx-master_db_kitchen           | Done
   mx    |           mx-master_db_industrial         | Done
   mx    |           mx-master_db_videogames         | Done
   mx    |              mx-master_db_toys            | Done
   mx    |             mx-master_db

In [ ]:
def centered_len(max_len):
    def centered_string(string):
        if type(string) != 'string':
            string = str(string)
            
        rest = max_len - len(string)
        n_spaces = int(rest/2)

        string_spaces = ' ' * n_spaces
        
        centered_string = string_spaces + string + string_spaces

        if len(centered_string)<max_len:
            centered_string = ' ' + centered_string

        return centered_string
    return centered_string